### TMA4320 - Project 2: Physics


# 0.0 Motivation

In [11]:
print("this is the second commit on the laptop-branch, from the large computer")



this is the second commit on the laptop-branch, from the large computer


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Parameters
sigma_0 = 1000 # kg/m^2
sigma = 500 # kg/m^2
R = 10 # m, Radius
A_s = 0.5*np.pi*(R**2) # Cross-section of boat

In [9]:
def merge_test():
    print("This is on tha master branch")
   


merge_test()





This is on tha master branch
This is not written on my laptop, but is still on the master branch
This is also not laptop, but a new commit


In [9]:
def merge_test():
    print("This is on thiucshguiscvgha master branch")

    print("This is not written on my laptop, but is still on the master branch")
    print("This is also not laptop, but a new commit")


    


merge_test()





This is on tha master branch
This is not written on my laptop, but is still on the master branch
This is also not laptop, but a new commit


In [ ]:
print("YO")
print("YOLO")

# X.0 Conclusion

Lorem Ipsum...